In [ ]:
import sys
sys.path.append('../')
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import json
import pytransform3d.transformations as pt
import pytransform3d.camera as pc
import pytransform3d.visualizer as pv
from data_tools.process_utils import fov2focal, generate_pcd

dataset = 'paris'
subset = 'sapien'
scenes = 'foldchair_102255 washer_103776 fridge_10905 blade_103706 storage_45135 oven_101917 stapler_103111 USB_100109 laptop_10211 scissor_11100'.split(' ')
# subset = 'realscan'
# scenes = 'real_fridge real_storage'.split(' ')
# scene = 'foldchair_102255'

# dataset = 'dta'
# subset = 'sapien'
# scenes = 'fridge_10489 storage_47254'.split(' ')
# scene = 'fridge_10489'

dataset = 'artgs'
subset = 'sapien'
scenes = 'oven_101908 table_25493 storage_45503 storage_47648 table_31249'.split(' ')
scene = 'oven_101908'

filename = f'../data/{dataset}/{subset}/{scene}/transforms_train.json'
info = json.load(open(filename, 'r'))

poses = [np.array(frame['transform_matrix'], np.float32) for frame in info['frames']]

w, h = Image.open(f'../data/{dataset}/{subset}/{scene}/start/train/rgba/0000.png').size
fovx, fovy = info['camera_angle_x'], info['camera_angle_y']
K = np.zeros((3, 3))
K[0, 0] = fov2focal(fovx, w)
K[1, 1] = fov2focal(fovy, h)
K[0, 2] = w // 2
K[1, 2] = h // 2
K[2, 2] = 1
sensor_size = (float(w), float(h))

fig = pv.figure()
mesh_filename = f'../data/{dataset}/{subset}/{scene}/gt/start/start_rotate.ply'
fig.plot_mesh(mesh_filename)

ill_poses = []
for i, pose in enumerate(poses):
    try:
        fig.plot_transform(A2B=pose, s=0.1)
        fig.plot_camera(M=K, cam2world=pose, virtual_image_distance=0.1, sensor_size=sensor_size)
    except:
        ill_poses.append((i, pose))
fig.show()